for i in ['enginetype','carbody','drivewheel','fuelsystem','car_company']:
    print(df_ia[i].value_counts(ascending=True))

from google.colab import files
for df in list_of_df:
  df.to_csv(df+f'{str}.csv', index = False, header=True) 
  files.download(df+'.csv')

# Start IA Regressions

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
import pandas as pd
df_immo = pd.read_csv('/content/gdrive/MyDrive/Datasets/immo_fr_simplon/valeursfoncieres-2020_loc.csv',low_memory=False)

Mounted at /content/gdrive


In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean.columns

Index(['Unnamed: 0', 'No disposition', 'Date mutation', 'Nature mutation',
       'Valeur fonciere', 'No voie', 'Type de voie', 'Code voie', 'Voie',
       'Code postal', 'Commune', 'Code departement', 'Code commune', 'Section',
       'No plan', '1er lot', 'Surface Carrez du 1er lot', 'Nombre de lots',
       'Code type local', 'Type local', 'Surface reelle bati',
       'Nombre pieces principales', 'Nature culture', 'Surface terrain',
       'latitude', 'longitude'],
      dtype='object')

In [ ]:
#OneHot
df_clean['Code type local'] = df_clean['Code type local'].fillna(0)
df_clean['Type local'] =df_clean['Type local'].fillna('none')
df_immo_full_cat = df_clean[['Type local']]
from sklearn.preprocessing import OneHotEncoder
cat_local = OneHotEncoder(handle_unknown='ignore')
#cat_local.fit(df_immo_full['Type local'].values.reshape(-1, 1))
df_immo_1hot = cat_local.fit_transform(df_immo_full_cat)
df_1hot = pd.DataFrame(df_immo_1hot.toarray(), columns = ['is_'+i for i in cat_local.categories_[0]])
df_immo_1hot = pd.concat([df_clean, df_1hot], axis=1)
df_immo_1hot.drop(columns = 'is_Local industriel. commercial ou assimilé', inplace = True)

### Overview2 only maison appart

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.inspection import permutation_importance

In [ ]:
def scores(model):
    y_train_predict = model.predict(X_test)
    rmse = (np.sqrt(mean_squared_error(y_test, y_test_predict)))
    r2 = r2_score(y_test, y_test_predict)
    mae = mean_absolute_error(y_test, y_test_predict)
    N = int(X_test.shape[0])
    p = int(X_test.shape[1])
    r2_adjusted = 1-((1-r2)*(N-1))/(N-p-1)
    return print('rmse is',rmse,'\nr2 is',r2,'\nmae is ', mae,'\nr2 ajusted is',r2_adjusted)

def zscore(df_col):
  zscore = (df_col - df_col.mean()) / df_col.std()
  return zscore

In [ ]:
templist = ['Maison','Appartement']
df_onlyma = df_immo_1hot[df_immo_1hot['Type local'].isin(templist)]
df_onlyma.shape

(915434, 30)

In [ ]:
reg_col = ['Valeur fonciere','Nombre de lots','Surface reelle bati',
  'Nombre pieces principales','Surface terrain','is_Maison','latitude', 'longitude']

In [ ]:
df_reg = df_onlyma[reg_col]

In [ ]:
my_zscore = zscore(df_reg['Valeur fonciere'])
df_reg.drop(my_zscore[(abs(my_zscore)>2)].index, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
X = df_reg.drop(columns='Valeur fonciere')
y = df_reg['Valeur fonciere']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state = 42)

## Qu'y a t'il dedans??

In [ ]:
df_reg

,Valeur fonciere,Nombre de lots,Surface reelle bati,Nombre pieces principales,Surface terrain,is_Maison,latitude,longitude
11,72000.0,0,35.0,2,381.0,1.0,46.386664,5.330823
13,180300.0,0,75.0,4,525.0,1.0,46.205152,5.246021
14,54800.0,2,32.0,1,0.0,0.0,46.205152,5.246021
16,350750.0,0,201.0,7,1267.0,1.0,46.211200,5.184628
17,350750.0,0,201.0,7,1497.0,1.0,46.211200,5.184628
...,...,...,...,...,...,...,...,...
2011820,1937500.0,0,29.0,2,447.0,0.0,48.854228,2.357362
2011821,1937500.0,0,27.0,2,447.0,0.0,48.854228,2.357362
2011822,1937500.0,0,87.0,4,447.0,0.0,48.854228,2.357362
2011824,1937500.0,0,82.0,2,447.0,0.0,48.854228,2.357362


In [ ]:
datas.to_csv('/content/gdrive/MyDrive/Datasets/immo_fr_simplon/mydf_w_loc.csv')

## Reg No Scaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state = 42)
reg=LinearRegression()
reg.fit(X_train,y_train)
y_test_predict = reg.predict(X_test)
scores(reg)

rmse is 218958.84418038727 
r2 is 0.10352417807754366 
mae is  135427.02157984863 
r2 ajusted is 0.10350301923866123


## Reg MinMax Scaler

In [ ]:
scaler = MinMaxScaler()
X_scale = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.33,random_state = 42)
reg=LinearRegression()
reg.fit(X_train,y_train)
y_test_predict = reg.predict(X_test)
scores(reg)

rmse is 218958.84418038742 
r2 is 0.10352417807754244 
mae is  135427.02157989412 
r2 ajusted is 0.10350301923865979


## Reg Standard Scaler

In [ ]:
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.33,random_state = 42)
reg=LinearRegression()
reg.fit(X_train,y_train)
y_test_predict = reg.predict(X_test)
scores(reg)

rmse is 184420.51369141781 
r2 is 0.05812995849715141 
mae is  124947.15004615704 
r2 ajusted is 0.04784432621864909


## Ridge Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

model = Ridge()
# define evaluation
cv = RepeatedKFold(n_splits=5, n_repeats=3)
# define search space
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
space['fit_intercept'] = [True, False]
space['normalize'] = [True, False]
# define search
search = GridSearchCV(model, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
# execute search
result = search.fit(X, y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: -132511.0595452173
Best Hyperparameters: {'alpha': 0.1, 'fit_intercept': True, 'normalize': True, 'solver': 'svd'}


In [ ]:
model = Ridge(alpha= 0.01, fit_intercept= True, normalize= True, solver= 'sag' )
model.fit(X_train,y_train)
y_test_predict = model.predict(X_test)
scores(model)

rmse is 218960.67189474247 
r2 is 0.10350921171641336 
mae is  135391.60187977343 
r2 ajusted is 0.10348805252429116


## XGBoost

In [ ]:
params = {'n_estimators': 500,
          'max_depth': 4 ,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
xg = ensemble.GradientBoostingRegressor(**params)
xg.fit(X_train, y_train)
y_test_predict = xg.predict(X_test)
scores(xg)

rmse is 177130.32482506166 
r2 is 0.41332249632722107 
mae is  102661.29197614559 
r2 ajusted is 0.4133086494230741


In [ ]:
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train,y_train)
y_test_predict = dummy_regr.predict(X_test)

In [ ]:
scores(dummy_regr)

rmse is 231256.169678657 
r2 is -5.836657011037971e-07 
mae is  145483.72558147408 
r2 ajusted is -2.4185921015495992e-05


# Add data

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
import pandas as pd
df_immo = pd.read_csv('/content/gdrive/MyDrive/Datasets/immo_fr_simplon/valeursfoncieres-2020_loc.csv',low_memory=False)
df_immo.drop(columns = 'Unnamed: 0',inplace = True,errors='ignore')

Mounted at /content/gdrive


In [30]:
df_immo.sort_values(['index','Date mutation', 'Commune','Code postal','Valeur fonciere'], ascending=[True,True, True,True, True])

KeyError: ignored

In [31]:
df_immo

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,latitude,longitude
0,1,2020-07-01,Vente,8000.0,NaN,NaN,B063,FORTUNAT,1250,CEYZERIAT,01,72,AK,216,NaN,NaN,0,0.0,none,0.0,0,T,1061.0,46.183217,5.320072
1,1,2020-02-01,Vente,2175.0,NaN,NaN,B124,TERRES DES CINQ SAULES,1290,LAIZ,01,203,B,4,NaN,NaN,0,0.0,none,0.0,0,BT,85.0,46.246992,4.899335
2,1,2020-02-01,Vente,2175.0,NaN,NaN,B006,BOIS DU CHAMP RION,1290,LAIZ,01,203,B,173,NaN,NaN,0,0.0,none,0.0,0,T,1115.0,46.246992,4.899335
3,1,2020-02-01,Vente,2175.0,NaN,NaN,B025,EN COROBERT,1290,LAIZ,01,203,B,477,NaN,NaN,0,0.0,none,0.0,0,T,1940.0,46.246992,4.899335
4,1,2020-02-01,Vente,2175.0,NaN,NaN,B124,TERRES DES CINQ SAULES,1290,LAIZ,01,203,C,68,NaN,NaN,0,0.0,none,0.0,0,T,1148.0,46.246992,4.899335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011822,1,2020-12-16,Vente,1937500.0,24.0,RUE,1748,CHANOINESSE,75004,PARIS 04,75,104,AW,19,NaN,NaN,0,2.0,Appartement,87.0,4,S,447.0,48.854228,2.357362
2011823,1,2020-12-16,Vente,1937500.0,24.0,RUE,1748,CHANOINESSE,75004,PARIS 04,75,104,AW,19,NaN,NaN,0,3.0,Dépendance,0.0,0,S,447.0,48.854228,2.357362
2011824,1,2020-12-16,Vente,1937500.0,24.0,RUE,1748,CHANOINESSE,75004,PARIS 04,75,104,AW,19,NaN,NaN,0,2.0,Appartement,82.0,2,S,447.0,48.854228,2.357362
2011825,1,2020-12-16,Vente,1937500.0,24.0,RUE,1748,CHANOINESSE,75004,PARIS 04,75,104,AW,19,NaN,NaN,0,3.0,Dépendance,0.0,0,S,447.0,48.854228,2.357362


In [18]:
df_immo.duplicated(subset=['Date mutation', 'Commune','Code postal','Valeur fonciere']).sum()

1031536

In [19]:
df_immo.head(50)

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,latitude,longitude
0,1,2020-07-01,Vente,8000.0,NaN,NaN,B063,FORTUNAT,1250,CEYZERIAT,01,72,AK,216,NaN,NaN,0,0.0,none,0.0,0,T,1061.0,46.183217,5.320072
1,1,2020-02-01,Vente,2175.0,NaN,NaN,B124,TERRES DES CINQ SAULES,1290,LAIZ,01,203,B,4,NaN,NaN,0,0.0,none,0.0,0,BT,85.0,46.246992,4.899335
2,1,2020-02-01,Vente,2175.0,NaN,NaN,B006,BOIS DU CHAMP RION,1290,LAIZ,01,203,B,173,NaN,NaN,0,0.0,none,0.0,0,T,1115.0,46.246992,4.899335
3,1,2020-02-01,Vente,2175.0,NaN,NaN,B025,EN COROBERT,1290,LAIZ,01,203,B,477,NaN,NaN,0,0.0,none,0.0,0,T,1940.0,46.246992,4.899335
4,1,2020-02-01,Vente,2175.0,NaN,NaN,B124,TERRES DES CINQ SAULES,1290,LAIZ,01,203,C,68,NaN,NaN,0,0.0,none,0.0,0,T,1148.0,46.246992,4.899335
5,1,2020-02-01,Vente,2175.0,NaN,NaN,B124,TERRES DES CINQ SAULES,1290,LAIZ,01,203,C,69,NaN,NaN,0,0.0,none,0.0,0,T,2960.0,46.246992,4.899335
6,1,2020-07-01,Vente,75000.0,NaN,RUE,0064,DE LA CHARTREUSE,1960,PERONNAS,01,289,AI,210,NaN,NaN,0,0.0,none,0.0,0,AB,610.0,46.171190,5.218022
7,1,2020-07-01,Vente,123.0,NaN,NaN,B041,CHAMP PORTIER,1370,VAL REVERMONT,01,426,ZN,197,NaN,NaN,0,0.0,none,0.0,0,S,55.0,46.280328,5.360832
8,1,2020-07-01,Vente,123.0,NaN,NaN,B041,CHAMP PORTIER,1370,VAL REVERMONT,01,426,ZN,201,NaN,NaN,0,0.0,none,0.0,0,S,68.0,46.280328,5.360832
9,1,2020-09-01,Vente,72000.0,NaN,NaN,B034,CHAMPEL,1270,COLIGNY,01,108,AC,15,NaN,NaN,0,0.0,none,0.0,0,J,328.0,46.386664,5.330823


In [27]:
df_immo[~df_immo['Nature culture'].isna()].shape

(1648773, 25)

In [28]:
df_immo[df_immo['Nature culture'].isna()].shape

(363054, 25)

In [42]:
import numpy as np
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6],[1, 2, 36], [7, 8, 9]]),

                   columns=['a', 'b', 'c'])
df2


,a,b,c
0,1,2,3
1,4,5,6
2,1,2,36
3,7,8,9


In [49]:
# Voila comment je vais recuperer toutes les lignes correspondant à une unique transaction pour de multiples biens
df = df2[df2.duplicated(subset=['a', 'b'], keep=False)]
print(df.index)
df

0


,a,b,c
0,1,2,3
2,1,2,36
